In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127991")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127991
Azure region: southcentralus
Subscription id: 2248b1c7-a065-4265-9476-038b58f0650e
Resource group: aml-quickstarts-127991


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "project-compute"

# if cluster already exists, use it
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster {} exists. Will use this cluster.'.format(cpu_cluster_name))
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## HyperDrive Run

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C': uniform(0, 1.0), 
                              '--max_iter': choice(10, 25, 50, 75, 100, 150, 250, 500, 1000)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
#policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
             compute_target=cpu_cluster_name,
             entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
assert(hyperdrive_run.get_status() == "Completed")

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()


In [10]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.9164214624777227', '--max_iter', '50']


In [9]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_460d3f530115f951f89932fb1105383bc7bbdf25b37ca13cbed32f518b996540_d.txt', 'azureml-logs/65_job_prep-tvmps_460d3f530115f951f89932fb1105383bc7bbdf25b37ca13cbed32f518b996540_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_460d3f530115f951f89932fb1105383bc7bbdf25b37ca13cbed32f518b996540_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_dda58004-8dcd-4179-9418-009c0f43055f.jsonl', 'logs/azureml/dataprep/python_span_l_dda58004-8dcd-4179-9418-009c0f43055f.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']


In [19]:
# model = best_run.register_model(model_name = 'best_HD_model', model_path = 'outputs/model.joblib')
# Note -- I am getting an error when trying to dump the best run: 
#       "Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run:"
# Should I be using a different file name from the get_file_names output?

In [17]:
best_run.get_metrics()

{'Regularization Strength:': 0.9164214624777227,
 'Max iterations:': 50,
 'Accuracy': 0.9146181082448154}

In [18]:
best_run.get_details()

{'runId': 'HD_fcc8a862-a012-4edb-a87e-68935d98f0cc_0',
 'target': 'project-compute',
 'status': 'Completed',
 'startTimeUtc': '2020-11-26T14:57:31.137556Z',
 'endTimeUtc': '2020-11-26T14:59:42.48719Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '52114e21-5df5-4de2-a2e2-f3b01fe54af0',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.9164214624777227', '--max_iter', '50'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'project-compute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'priority': None,
  'environment': {'name': 'Experiment udacity-project Environment',
   'version': 'Autosave_2020-11-26T14:47:16Z_9

## AutoML

In [24]:
from azureml.data.dataset_factory import TabularDatasetFactory as tdf

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = tdf.from_delimited_files(path=data_path)

In [25]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
train_df = x.copy()
train_df['target'] = y

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_df,
    label_column_name='target',
    n_cross_validations=5)

In [27]:
# Submit your automl run

automl_exp = Experiment(ws, "automl_experiment")
automl_run = automl_exp.submit(config=automl_config, show_output=True)

WARNING - Connection pool is full, discarding connection: southcentralus.api.azureml.ms


Running on local machine


WARNING - Connection pool is full, discarding connection: southcentralus.api.azureml.ms


Parent Run ID: AutoML_5ed590c9-6f81-4670-b697-a76a36ed1226

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect 

In [34]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_aml_run = automl_run.get_best_child()

In [29]:
best_aml_run.get_metrics('accuracy')

{'accuracy': 0.9163277693474962}

In [30]:
best_aml_run.get_details()

{'runId': 'AutoML_5ed590c9-6f81-4670-b697-a76a36ed1226_22',
 'status': 'Completed',
 'startTimeUtc': '2020-11-25T23:45:20.26894Z',
 'endTimeUtc': '2020-11-25T23:51:48.172162Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl_experiment\',\'compute_target\':\'local\',\'subscription_id\':\'422c78cb-7b9f-41e7-a94c-e144f7afde5e\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_5ed590c9-6f81-4670-b697-a76a36ed1226_22","experiment_name":null,"workspace_name":"quick-starts-ws-127925","subscription_id":"422c78cb-7b9f-41e7-a94c-e144f7afde5e","resource_group_name":"aml-qu

## Delete Compute Cluster

In [33]:
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

